# 코로나 감염 시·도발생 현황 API

- 보건복지부
- https://www.data.go.kr/iim/api/selectAPIAcountView.do

- 코로나19감염증으로 인한 시.도별 신규확진자,신규사망자,격리중인환자수,격리해제환자수등에 대한 현황자료

- 03.보건복지부_OpenAPI활용가이드_코로나바이러스감염증_시도발생_현황 조회 서비스_v0.3 (2).docx 참고 하여 작성


- 날짜 / 시도명 / 확진자 수 / 지역발생 수

- resultCode	결과코드
- resultMsg	결과메세지
- numOfRows	한 페이지 결과 수
- pageNo	페이지 수
- totalCount	전체 결과 수
- SEQ	게시글번호(국내 시도별 발생현황 고유값)
- CREATE_DT 등록일시분초
- DEATH_CNT 사망자 수
- **GUBUN 시도명(한글)**
- GUBUN_CN 시도명(중국어)
- GUBUN_EN 시도명(영어)
- INC_DEC 전일대비 증감수
- ISOL_CLEAR_CNT 격리 해제 수
- QUR_RATE 10만명당 발생률
- **STD_DAY 기준일시**
- UPDATE_DT 수정일시분초
- **DEF_CNT 확진자 수** : 누적 확진자 수
- ISOL_ING_CNT 격리중 환자수
- OVER_FLOW_CNT 해외유입 수
- **LOCAL_OCC_CNT 지역발생 수** : 일별 확진자 수


## 데이터 불러오기

In [1]:
import urllib.request
import json
import pandas as pd
from datetime import datetime,timedelta
import requests
import xmltodict

## 코로나19감염증으로 인한 일별 확진자, 완치자, 치료중인환자, 사망자등에 대한 현황자료
url_base = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19SidoInfStateJson'
url_serviceKey = 'zDi7%2BxdkbZUvy%2Bw2jREuuky26qthb6FliCJd4uJTY8qOo02u2VFch2k2zvE0GKtsF8nLCKxd7WqoNvU%2BuvCaUQ%3D%3D'
url_numOfRows = '1000'
url_start_date = '20200120'
url_end_date = datetime.today().strftime('%Y%m%d') # 현재시간 기준으로 데이터 불러오기

# Call Back URL + 인증키 +  페이지 번호(= 1) + 한 페이지 결과 수 + 데이터 생성일 시작 범위 + 데이터 생성일 종료 범위
url = url_base + '?serviceKey=' + url_serviceKey + '&pageNo=1' + '&num0fRows=' + url_numOfRows + '&startCreateDt=' + url_start_date + '&endCreateDt=' + url_end_date
url

'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19SidoInfStateJson?serviceKey=zDi7%2BxdkbZUvy%2Bw2jREuuky26qthb6FliCJd4uJTY8qOo02u2VFch2k2zvE0GKtsF8nLCKxd7WqoNvU%2BuvCaUQ%3D%3D&pageNo=1&num0fRows=1000&startCreateDt=20200120&endCreateDt=20211130'

In [2]:
req = requests.get(url, allow_redirects=False).content

xmlObject = xmltodict.parse(req)

ExpatError: not well-formed (invalid token): line 1, column 102

In [37]:
dict_data = xmlObject['response']['body']['items']['item']

df = pd.DataFrame(dict_data)

In [38]:
pd.isna(df).sum()

createDt        0
deathCnt        0
defCnt          0
gubun           0
gubunCn         0
gubunEn         0
incDec          0
isolClearCnt    0
isolIngCnt      0
localOccCnt     0
overFlowCnt     0
qurRate         0
seq             0
stdDay          0
updateDt        0
dtype: int64

## 필요한 열만 남겨두기 & 열 이름 변경 

In [39]:
df_conf = df.copy()

In [40]:
# 날짜, 시도명(한글), 확진자 수, 지역발생 수
# stdDay, gubun, defCnt, localOccCnt
df_conf = df_conf[['stdDay','gubun','defCnt','localOccCnt']]

df_conf.columns = ['날짜','시도명','확진자 수','지역발생 수']
df_conf

,날짜,시도명,확진자 수,지역발생 수
0,2021년 11월 30일 00시,검역,6449,0
1,2021년 11월 30일 00시,제주,3704,21
2,2021년 11월 30일 00시,경남,15015,60
3,2021년 11월 30일 00시,경북,11268,68
4,2021년 11월 30일 00시,전남,4419,35
...,...,...,...,...
12915,2020년 01월 20일 00시,전남,0,0
12916,2020년 01월 20일 00시,경북,0,0
12917,2020년 01월 20일 00시,경남,0,0
12918,2020년 01월 20일 00시,제주,0,0


## 시도명 으로 묶기

In [47]:
# df_conf = df_conf[''].groupby(df_conf[''])
grouped = df_conf.groupby(['시도명', '날짜'], as_index=False).sum()
grouped

,시도명,날짜,확진자 수,지역발생 수
0,강원,2020년 01월 20일 00시,0,0
1,강원,2020년 01월 21일 00시,0,0
2,강원,2020년 01월 22일 00시,0,0
3,강원,2020년 01월 23일 00시,0,0
4,강원,2020년 01월 24일 00시,0,0
...,...,...,...,...
12915,합계,2021년 11월 26일 00시,432901,3882
12916,합계,2021년 11월 27일 00시,436968,4045
12917,합계,2021년 11월 28일 00시,440896,3893
12918,합계,2021년 11월 29일 00시,444200,3286


In [60]:
합계_df = grouped[grouped['시도명']=='합계']
합계_df

,시도명,날짜,확진자 수,지역발생 수
12240,합계,2020년 01월 20일 00시,1,0
12241,합계,2020년 01월 21일 00시,1,0
12242,합계,2020년 01월 22일 00시,1,0
12243,합계,2020년 01월 23일 00시,1,0
12244,합계,2020년 01월 24일 00시,2,0
...,...,...,...,...
12915,합계,2021년 11월 26일 00시,432901,3882
12916,합계,2021년 11월 27일 00시,436968,4045
12917,합계,2021년 11월 28일 00시,440896,3893
12918,합계,2021년 11월 29일 00시,444200,3286


## 일일 확진자 도시별로 시트별로 저장

In [58]:
# 유일한 값 찾기
city_list = grouped['시도명'].unique().tolist()
city_list

['강원',
 '검역',
 '경기',
 '경남',
 '경북',
 '광주',
 '대구',
 '대전',
 '부산',
 '서울',
 '세종',
 '울산',
 '인천',
 '전남',
 '전북',
 '제주',
 '충남',
 '충북',
 '합계']

In [63]:
df_list = []

for city in city_list:
    df_list.append(grouped[grouped['시도명']==city])

In [68]:
df_list

[    시도명                 날짜 확진자 수 지역발생 수
 0    강원  2020년 01월 20일 00시     0      0
 1    강원  2020년 01월 21일 00시     0      0
 2    강원  2020년 01월 22일 00시     0      0
 3    강원  2020년 01월 23일 00시     0      0
 4    강원  2020년 01월 24일 00시     0      0
 ..   ..                ...   ...    ...
 675  강원  2021년 11월 26일 00시  8344     53
 676  강원  2021년 11월 27일 00시  8437     93
 677  강원  2021년 11월 28일 00시  8500     61
 678  강원  2021년 11월 29일 00시  8556     55
 679  강원  2021년 11월 30일 00시  8640     84
 
 [680 rows x 4 columns],
      시도명                 날짜 확진자 수 지역발생 수
 680   검역  2020년 01월 20일 00시     0      0
 681   검역  2020년 01월 21일 00시     0      0
 682   검역  2020년 01월 22일 00시     0      0
 683   검역  2020년 01월 23일 00시     0      0
 684   검역  2020년 01월 24일 00시     0      0
 ...   ..                ...   ...    ...
 1355  검역  2021년 11월 26일 00시  6423      0
 1356  검역  2021년 11월 27일 00시  6428      0
 1357  검역  2021년 11월 28일 00시  6438      0
 1358  검역  2021년 11월 29일 00시  6441      0
 1359  검역  2021년 11

In [61]:
import datetime as dt
from datetime import datetime, timedelta

def get_date(y, m, d):
    s = f'{y:04d}-{m:02d}-{d:02d}'
    return datetime.strptime(s, '%Y-%m-%d')

def get_week_no(y, m, d):
    target = get_date(y, m, d)
    firstday = target.replace(day = 1)
    if firstday.weekday() == 6:
        origin = firstday
    elif firstday.weekday() < 3:
        origin = firstday - timedelta(days=firstday.weekday() + 1)
    else:
        origin = firstday + timedelta(days = 6-firstday.weekday())
    return (target - origin).days // 7 + 1

In [62]:
n = dt.datetime.now()

writer = pd.ExcelWriter('국내 코로나 확진자-지역별 일일 현황_{}년 {}월 {}주차.xlsx'
                        .format(n.year, n.month, get_week_no(n.year, n.month, n.day)), 
                        engine = 'xlsxwriter')

In [1]:
a =1 

In [71]:
for i, df in enumerate(df_list):
    print(city_list[i])
    print(df)
    # df.to_excel(writer, index = False, sheet_name = city_list[i])
    
# writer.save()

강원
    시도명                 날짜 확진자 수 지역발생 수
0    강원  2020년 01월 20일 00시     0      0
1    강원  2020년 01월 21일 00시     0      0
2    강원  2020년 01월 22일 00시     0      0
3    강원  2020년 01월 23일 00시     0      0
4    강원  2020년 01월 24일 00시     0      0
..   ..                ...   ...    ...
675  강원  2021년 11월 26일 00시  8344     53
676  강원  2021년 11월 27일 00시  8437     93
677  강원  2021년 11월 28일 00시  8500     61
678  강원  2021년 11월 29일 00시  8556     55
679  강원  2021년 11월 30일 00시  8640     84

[680 rows x 4 columns]
검역
     시도명                 날짜 확진자 수 지역발생 수
680   검역  2020년 01월 20일 00시     0      0
681   검역  2020년 01월 21일 00시     0      0
682   검역  2020년 01월 22일 00시     0      0
683   검역  2020년 01월 23일 00시     0      0
684   검역  2020년 01월 24일 00시     0      0
...   ..                ...   ...    ...
1355  검역  2021년 11월 26일 00시  6423      0
1356  검역  2021년 11월 27일 00시  6428      0
1357  검역  2021년 11월 28일 00시  6438      0
1358  검역  2021년 11월 29일 00시  6441      0
1359  검역  2021년 11월 30일 00시  6449      

In [6]:
# 공공데이터포털의 일일값의 합과 누적값에 차이 있어
# 명확한 가이드라인이 주어지지 않으면 누적값을 차분에 계산

# 숫자여야 할 열(누적확진자)이 object로 되어있으므로 숫자로 형변환 필요
df_conf_1.iloc[:,7] = df_conf_1.iloc[:,7].apply(pd.to_numeric)

# 누적확진자를 일일확진자로 변경
# 전날 decideCnt - 당일 decideCnt = 당일 확진자 수
df_conf_1['daily_decideCnt'] = df_conf_1['decideCnt'].diff()


# 숫자여야 할 열(누적 사망자 수)이 object로 되어있으므로 숫자로 형변환
df_conf_1.iloc[:,6] = df_conf_1.iloc[:,6].apply(pd.to_numeric)

# 누적 사망자를 일일 사망자로 변경
# 전날 deathCnt - 당일 deathCnt = 당일 사망자 수
df_conf_1['daily_deathCnt'] = df_conf_1['deathCnt'].diff()


# 숫자여야 할 열(누적검사수)이 object로 되어있으므로 숫자로 형변환 필요
df_conf_1.iloc[:,1] = df_conf_1.iloc[:,1].apply(pd.to_numeric)
# 누적검사수를 일일검사수로 변경
df_conf_1['daily_ExamCnt'] = df_conf_1['accExamCnt'].diff()

# 날짜, 확진자 수, 누적 확진자 수, 사망자 수, 누적 사망자 수, 검사자 수, 누적 검사자 수
# 1차 백신 접종자 수, 누적 1차 백신 접종자 수, 2차 백신 접종자 수, 누적 2차 백신 접종자 수
df_conf_2 = df_conf_1[['stateDt','daily_decideCnt','decideCnt','daily_deathCnt','deathCnt','daily_ExamCnt','accExamCnt']]

df_conf_2.columns = ['날짜','확진자 수','누적 확진자 수','사망자 수','누적 사망자 수','검사자 수','누적 검사자 수']

In [7]:
df_conf_2

,날짜,확진자 수,누적 확진자 수,사망자 수,누적 사망자 수,검사자 수,누적 검사자 수
674,20200120,NaN,1,NaN,0,NaN,4
673,20200121,0.0,1,0.0,0,11.0,15
672,20200122,0.0,1,0.0,0,6.0,21
671,20200123,0.0,1,0.0,0,4.0,25
670,20200124,1.0,2,0.0,0,4.0,29
...,...,...,...,...,...,...,...
4,20211121,3120.0,415423,30.0,3274,37560.0,16777680
3,20211122,2827.0,418250,24.0,3298,38937.0,16816617
2,20211123,2699.0,420949,30.0,3328,62341.0,16878958
1,20211124,4115.0,425064,34.0,3362,60970.0,16939928


In [8]:
df_conf_2.loc[df_conf_2['날짜'] == '20200120', '확진자 수'] = '1'
df_conf_2

C:\Users\user\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,날짜,확진자 수,누적 확진자 수,사망자 수,누적 사망자 수,검사자 수,누적 검사자 수
674,20200120,1,1,NaN,0,NaN,4
673,20200121,0.0,1,0.0,0,11.0,15
672,20200122,0.0,1,0.0,0,6.0,21
671,20200123,0.0,1,0.0,0,4.0,25
670,20200124,1.0,2,0.0,0,4.0,29
...,...,...,...,...,...,...,...
4,20211121,3120.0,415423,30.0,3274,37560.0,16777680
3,20211122,2827.0,418250,24.0,3298,38937.0,16816617
2,20211123,2699.0,420949,30.0,3328,62341.0,16878958
1,20211124,4115.0,425064,34.0,3362,60970.0,16939928


In [9]:
df_conf_2.head(50)

,날짜,확진자 수,누적 확진자 수,사망자 수,누적 사망자 수,검사자 수,누적 검사자 수
674,20200120,1,1,NaN,0,NaN,4
673,20200121,0.0,1,0.0,0,11.0,15
672,20200122,0.0,1,0.0,0,6.0,21
671,20200123,0.0,1,0.0,0,4.0,25
670,20200124,1.0,2,0.0,0,4.0,29
669,20200125,0.0,2,0.0,0,18.0,47
668,20200126,1.0,3,0.0,0,9.0,56
667,20200127,1.0,4,0.0,0,28.0,84
666,20200128,0.0,4,0.0,0,53.0,137
665,20200129,0.0,4,0.0,0,71.0,208


In [10]:
df_conf_2.tail(20)

,날짜,확진자 수,누적 확진자 수,사망자 수,누적 사망자 수,검사자 수,누적 검사자 수
19,20211106,2246.0,377708,20.0,2956,47518.0,16099185
18,20211107,2225.0,379933,11.0,2967,31687.0,16130872
17,20211108,1754.0,381687,13.0,2980,29956.0,16160828
16,20211109,1717.0,383404,18.0,2998,52042.0,16212870
15,20211110,2425.0,385829,14.0,3012,49007.0,16261877
14,20211111,2518.0,388347,21.0,3033,50894.0,16312771
13,20211112,2368.0,390715,18.0,3051,48257.0,16361028
12,20211113,2324.0,393039,32.0,3083,47570.0,16408598
11,20211114,2418.0,395457,20.0,3103,31829.0,16440427
10,20211115,2004.0,397461,12.0,3115,30542.0,16470969


## 시트별로 저장하기

- 현재 날짜에 맞춰 '2021년  N월 M주차_국내 코로나 확진자-백신접종자 현황'으로 저장하기


In [61]:
import datetime as dt
from datetime import datetime, timedelta

def get_date(y, m, d):
    s = f'{y:04d}-{m:02d}-{d:02d}'
    return datetime.strptime(s, '%Y-%m-%d')

def get_week_no(y, m, d):
    target = get_date(y, m, d)
    firstday = target.replace(day = 1)
    if firstday.weekday() == 6:
        origin = firstday
    elif firstday.weekday() < 3:
        origin = firstday - timedelta(days=firstday.weekday() + 1)
    else:
        origin = firstday + timedelta(days = 6-firstday.weekday())
    return (target - origin).days // 7 + 1

In [62]:
n = dt.datetime.now()

writer = pd.ExcelWriter('국내 코로나 확진자-지역별 일일 현황_{}년 {}월 {}주차.xlsx'
                        .format(n.year, n.month, get_week_no(n.year, n.month, n.day)), 
                        engine = 'xlsxwriter')

In [125]:
for i, df in enumerate(df_list):
    df.to_excel(writer, index = False, sheet_name = region[i])
    
writer.save()